In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [3]:
import numpy as np

In [4]:
import pandas as pd

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_97749/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Data Preprocessing

### 1. Book-Crossing Dataset

In [6]:
# Reading the CSV files and skipping the bad lines
books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
ratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Book-Ratings.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
users = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Users.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_97749/3698374196.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')


In [7]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### 1.1 Books Table

In [8]:
#Renaming the columns
books.rename(columns={'Book-Title': 'Title', 'Book-Author': 'Author', 'Year-Of-Publication': 'Year'}, inplace=True)

In [9]:
#Dropping the URL columns for the Books table
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

In [10]:
books.loc[books['Author'].isnull(), 'Author'] = "Others"

In [11]:
books.loc[books['Publisher'].isnull(), 'Publisher'] = "Others"

In [12]:
books = books.drop_duplicates()

In [13]:
books.at[209538, 'Publisher'] = 'DK Publishing Inc'
books.at[209538, 'Year'] = 2000
books.at[209538, 'Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books.at[209538, 'Author'] = 'Michael Teitelbaum'

In [14]:
books.at[221678, 'Publisher'] = 'DK Publishing Inc'
books.at[221678, 'Year'] = 2000
books.at[221678, 'Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books.at[221678, 'Author'] = 'James Buckley'

In [15]:
books.at[220731, 'Publisher'] = 'Gallimard'
books.at[220731, 'Year'] = 2003
books.at[220731, 'Title'] = 'Peuple du ciel - Suivi de Les bergers '
books.at[220731, 'Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [16]:
#converting year to integer values
books['Year'] = pd.to_numeric(books['Year'], errors='coerce')

In [17]:
count = books['Year'].value_counts()
most_commen_year = count[count == count.max()].index.tolist()[0]

In [18]:
#Replacing all values for Publishing years greater than 2004 or equal to 0 with the maximum year
books.loc[books['Year'] > 2004, 'Year'] = most_commen_year

In [19]:
books.loc[books['Year'] == 0, 'Year'] = most_commen_year

### 1.2 Users Table

In [20]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [21]:
#Renaming the columns
users.rename(columns={'User-ID': 'UserID'}, inplace=True)

#### Ages that have values nan, less than 8, and more than 90 are invalid 

In [22]:
age_range = users[users['Age'] <= 90] 
age_range = age_range[age_range['Age'] >= 8]
mean = age_range['Age'].mean().round()   

In [23]:
users.loc[(users['Age'] > 90) | (users['Age'] < 8),'Age'] = mean
users['Age']=users['Age'].fillna(mean).astype(int)

### 1.3 Ratings Table

In [24]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [25]:
#Renaming the columns
ratings.rename(columns={'User-ID': 'UserID', 'Book-Rating': 'BookRating'}, inplace=True)

In [26]:
# Assuming 'df' is your original DataFrame
bxratings_copy = ratings.copy()
bxratings_copy.head()

,UserID,ISBN,BookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [27]:
# bx_userratecopy = bxratings_copy.groupby(['ISBN'], as_index=False)['BookRating'].mean().round(0)
# bx_userratecopy.value_counts()
bx_userratecopy = bxratings_copy.groupby(['ISBN']).agg({'UserID': 'first', 'BookRating': 'mean'}).reset_index()
bxratings_copy.head()

,UserID,ISBN,BookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [28]:
#Scaling the ratings to from a scale of 1-10 to 1-5
scaled_ratingscopy = (bx_userratecopy['BookRating'] - 1) * (5 / 9) + 1
bx_userratecopy['Scaled Rating'] = np.clip(round(scaled_ratingscopy), 1, 5)
bx_userratecopy['Scaled Rating'].unique()

array([2., 1., 3., 5., 4.])

In [29]:
bx_userratecopy['Scaled Rating'] = bx_userratecopy['Scaled Rating'].astype(int)
bx_userratecopy.head()

,ISBN,UserID,BookRating,Scaled Rating
0,0330299891,89192,3.0,2
1,0375404120,89192,1.5,1
2,0586045007,89192,0.0,1
3,9022906116,11676,3.5,2
4,9032803328,89192,0.0,1


In [30]:
ratings = ratings.drop(["UserID"], axis=1)

In [31]:
bx_userrate = ratings.groupby(['ISBN'], as_index=False)['BookRating'].mean().round(0)
bx_userrate.value_counts()

ISBN         BookRating
 0330299891  3.0           1
0875961096   0.0           1
0875961010   0.0           1
087595247X   0.0           1
0875952372   10.0          1
                          ..
0451124847   0.0           1
0451124693   0.0           1
0451124685   2.0           1
0451124634   0.0           1
Ô½crosoft    7.0           1
Name: count, Length: 340556, dtype: int64

In [32]:
#Scaling the ratings to from a scale of 1-10 to 1-5
scaled_ratings = (bx_userrate['BookRating'] - 1) * (5 / 9) + 1
bx_userrate['Scaled Rating'] = np.clip(round(scaled_ratings), 1, 5)
bx_userrate['Scaled Rating'].unique()

array([2., 1., 3., 5., 4.])

In [33]:
bx_userrate['Scaled Rating'] = bx_userrate['Scaled Rating'].astype(int)
bx_userrate.head()

,ISBN,BookRating,Scaled Rating
0,0330299891,3.0,2
1,0375404120,2.0,2
2,0586045007,0.0,1
3,9022906116,4.0,3
4,9032803328,0.0,1


### 1.4 Merging Books, Users, and Ratings tables

In [34]:
bx = pd.merge(books, bx_userrate, on='ISBN')
bx.head()

,ISBN,Title,Author,Year,Publisher,BookRating,Scaled Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0.0,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,5.0,3
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,5.0,3
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,4.0,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0.0,1


In [35]:
id_bx = pd.merge(books, bx_userratecopy, on='ISBN')
id_bx.head()

,ISBN,Title,Author,Year,Publisher,UserID,BookRating,Scaled Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0.000000,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,4.928571,3
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,5.000000,3
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,4.272727,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0.000000,1


### 2. Goodbooks-10k Dataset

In [36]:
goodbooks = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/books.csv', sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/ratings.csv', sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodbooks_tags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/book_tags.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodtags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/tags.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')
good_toread = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/to_read.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')

### 2.1 Books Table

In [37]:
#Dropping the URL columns for the Books table
goodbooks.drop(['image_url', 'small_image_url'], axis=1, inplace=True)

In [38]:
#Renaming the columns
goodbooks.rename(columns={'book_id': 'BookID', 'isbn': 'ISBN', 'authors': 'Author', 'original_publication_year': 'Year', 'original_title': 'Title', 'average_rating': 'Avgrating'}, inplace=True)

In [39]:
average_year = goodbooks.loc[goodbooks['Year'] >= 1800, 'Year'].mean().round()
average_year

1994.0

In [40]:
goodbooks.loc[goodbooks['Year'] == 0, 'Year'] = average_year
goodbooks.loc[:, 'Year'] = goodbooks['Year'].fillna(average_year)
goodbooks.loc[goodbooks['Year'] < 1800, 'Year'] = average_year

In [41]:
goodbooks.dropna(subset=['Title'], inplace=True)

In [42]:
goodbooks.shape

(9415, 21)

In [43]:
goodbooks['Year'] = goodbooks['Year'].astype(int)

In [44]:
#creating a new data frame only with the relevant columns
selected_cols = ['id', 'BookID', 'Author', 'Year', 'Title', 'Avgrating', 'ratings_count']
new_goodbooks = goodbooks[selected_cols].copy()

In [45]:
new_goodbooks.shape

(9415, 7)

### 2.2 Ratings Table

In [46]:
#Renaming the columns
goodratings.rename(columns={'book_id': 'BookID', 'user_id': 'UserID', 'rating': 'Rating'}, inplace=True)

In [47]:
# goodratings_copy = goodratings.copy()

In [48]:
user_Ratecopy = goodratings.groupby(['BookID']).agg({'UserID': 'first', 'Rating': 'mean'}).reset_index()
user_Ratecopy['Rating'] = user_Ratecopy['Rating'].round(0)
user_Ratecopy.rename(columns={'Rating': 'Mean Rating'}, inplace=True)
user_Ratecopy.head()

,BookID,UserID,Mean Rating
0,1,314,4.0
1,2,1169,4.0
2,3,314,3.0
3,4,439,4.0
4,5,314,4.0


In [49]:
user_Ratecopy['Mean Rating'] = user_Ratecopy['Mean Rating'].astype(int)
user_Ratecopy.head()

,BookID,UserID,Mean Rating
0,1,314,4
1,2,1169,4
2,3,314,3
3,4,439,4
4,5,314,4


In [50]:
user_rate = goodratings.groupby(['BookID'], as_index=False)['Rating'].mean().round(0)
user_rate.head()
# user_rate.value_counts()

,BookID,Rating
0,1,4.0
1,2,4.0
2,3,3.0
3,4,4.0
4,5,4.0


In [51]:
user_rate.shape

(10000, 2)

In [52]:
#Renaming the columns
user_rate.rename(columns={'Rating': 'Mean Rating'}, inplace=True)
user_rate.head()
# user_rate.value_counts()

,BookID,Mean Rating
0,1,4.0
1,2,4.0
2,3,3.0
3,4,4.0
4,5,4.0


In [53]:
user_rate['Mean Rating'] = user_rate['Mean Rating'].astype(int)

In [54]:
user_rate.shape

(10000, 2)

### 2.6 Merging the Books and Ratings tables

In [55]:
new_goodbooks

,id,BookID,Author,Year,Title,Avgrating,ratings_count
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664
...,...,...,...,...,...,...,...
9995,9996,7130616,Ilona Andrews,2010,Bayou Moon,4.09,17204
9996,9997,208324,Robert A. Caro,1990,Means of Ascent,4.25,12582
9997,9998,77431,Patrick O'Brian,1977,The Mauritius Command,4.35,9421
9998,9999,8565083,Peggy Orenstein,2011,Cinderella Ate My Daughter: Dispatches from th...,3.65,11279


In [56]:
user_Ratecopy = user_Ratecopy.rename(columns={"BookID":"id"})
user_Ratecopy.head()

,id,UserID,Mean Rating
0,1,314,4
1,2,1169,4
2,3,314,3
3,4,439,4
4,5,314,4


In [57]:
user_rate = user_rate.rename(columns={"BookID":"id"})

In [58]:
user_rate.head()

,id,Mean Rating
0,1,4
1,2,4
2,3,3
3,4,4
4,5,4


In [59]:
gbx = pd.merge(new_goodbooks, user_rate, on='id')

In [60]:
id_gbx = pd.merge(new_goodbooks, user_Ratecopy, on='id')

In [61]:
gbx.shape

(9415, 8)

In [62]:
id_gbx.shape

(9415, 9)

In [63]:
gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,Mean Rating
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,4
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,4
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,3
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,4
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,4


In [64]:
id_gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,UserID,Mean Rating
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,314,4
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,1169,4
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,314,3
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,439,4
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,314,4


## NEW

In [65]:
import nltk #using the nltk library for tokenization and lemmatization
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
#importing necessary libraries
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [67]:
#Function to clean text data
def clean_text(text):
    text = text.lower()     #Converting the text to lowercase
    text = re.sub(r'\d+', ' ', text)    # Removing digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Removing special characters
    text = text.translate(str.maketrans('', '', string.punctuation))    #Removing punctuation     
    tokens = word_tokenize(text) #Tokenization of the text
    non_stopwords = []  #Creating an empty list to store the non-stopwords
    stop_words = set(stopwords.words('english')) #Initializing the NLTK English stopwords
    #Lemmatization of the words/tokens
    lemmatizer = WordNetLemmatizer()    #Initialization of the NLTK WordNet Lemmatizer

    #Iterating through the list of tokens, lemmatizing them, and adding it to the list of non-stopwords
    for token in tokens:
        if token not in stop_words:
            lemma = lemmatizer.lemmatize(token) #Performing lemmatization
            non_stopwords.append(lemma)  # Appending the stemmed token back to the list of non-stopwords

    # Joining the tokens/words from the list of non-stopwords into a string
    filtered_text = ' '.join(non_stopwords)
    return filtered_text


In [68]:
# Apply the text_clean function to the 'Title' column of the first data frame 'bx' i.e., the Book Crossing dataset's final data frame
bx['cleaned_title'] = bx['Title'].apply(clean_text)

In [69]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - bx:")
print(bx[['Title', 'cleaned_title']].head())

First DataFrame - bx:
                                               Title  \
0                                Classical Mythology   
1                                       Clara Callan   
2                               Decision in Normandy   
3  Flu: The Story of the Great Influenza Pandemic...   
4                             The Mummies of Urumchi   

                                       cleaned_title  
0                                classical mythology  
1                                       clara callan  
2                                  decision normandy  
3  flu story great influenza pandemic search viru...  
4                                      mummy urumchi  


In [70]:
# Apply the text_clean function to the 'Title' column of the second data frame 'gbx' i.e., the Goodbooks-10k dataset's final data frame
gbx['cleaned_title'] = gbx['Title'].apply(clean_text)

In [71]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - gbx:")
print(gbx[['Title', 'cleaned_title']].head())


Second DataFrame - gbx:
                                      Title                   cleaned_title
0                          The Hunger Games                     hunger game
1  Harry Potter and the Philosopher's Stone  harry potter philosopher stone
2                                  Twilight                        twilight
3                     To Kill a Mockingbird                kill mockingbird
4                          The Great Gatsby                    great gatsby


### Cleaning 'Author' columns

In [72]:
# Apply the clean_text function to the 'Author' column of the first data frame 'bx'
bx['cleaned_author'] = bx['Author'].apply(clean_text)

In [73]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - bx:")
print(bx[['Author', 'cleaned_author']].head())

First DataFrame - bx:
                 Author        cleaned_author
0    Mark P. O. Morford        mark p morford
1  Richard Bruce Wright  richard bruce wright
2          Carlo D'Este           carlo deste
3      Gina Bari Kolata      gina bari kolata
4       E. J. W. Barber          e j w barber


In [74]:
# Apply the clean_text function to the 'Author' column of the second data frame 'gbx'
gbx['cleaned_author'] = gbx['Author'].apply(clean_text)

In [75]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - gbx:")
print(gbx[['Author', 'cleaned_author']].head())


Second DataFrame - gbx:
                         Author           cleaned_author
0               Suzanne Collins          suzanne collins
1  J.K. Rowling, Mary GrandPrÃ©  jk rowling mary grandpr
2               Stephenie Meyer          stephenie meyer
3                    Harper Lee               harper lee
4           F. Scott Fitzgerald       f scott fitzgerald


In [76]:
bx.head()

,ISBN,Title,Author,Year,Publisher,BookRating,Scaled Rating,cleaned_title,cleaned_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0.0,1,classical mythology,mark p morford
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,5.0,3,clara callan,richard bruce wright
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,5.0,3,decision normandy,carlo deste
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,4.0,3,flu story great influenza pandemic search viru...,gina bari kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,0.0,1,mummy urumchi,e j w barber


In [77]:
bxcolumns_to_copy = ['ISBN', 'Publisher', 'BookRating']

In [78]:
bx.drop(columns=bxcolumns_to_copy, inplace=True)
bx.head()

,Title,Author,Year,Scaled Rating,cleaned_title,cleaned_author
0,Classical Mythology,Mark P. O. Morford,2002,1,classical mythology,mark p morford
1,Clara Callan,Richard Bruce Wright,2001,3,clara callan,richard bruce wright
2,Decision in Normandy,Carlo D'Este,1991,3,decision normandy,carlo deste
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,3,flu story great influenza pandemic search viru...,gina bari kolata
4,The Mummies of Urumchi,E. J. W. Barber,1999,1,mummy urumchi,e j w barber


In [79]:
gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,Mean Rating,cleaned_title,cleaned_author
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,4,hunger game,suzanne collins
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,4,harry potter philosopher stone,jk rowling mary grandpr
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,3,twilight,stephenie meyer
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,4,kill mockingbird,harper lee
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,4,great gatsby,f scott fitzgerald


In [80]:
newgbxcolumns_to_copy = ['id','BookID', 'Avgrating', 'ratings_count']

In [81]:
gbx.drop(columns=newgbxcolumns_to_copy, inplace=True)
gbx.head()

,Author,Year,Title,Mean Rating,cleaned_title,cleaned_author
0,Suzanne Collins,2008,The Hunger Games,4,hunger game,suzanne collins
1,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4,harry potter philosopher stone,jk rowling mary grandpr
2,Stephenie Meyer,2005,Twilight,3,twilight,stephenie meyer
3,Harper Lee,1960,To Kill a Mockingbird,4,kill mockingbird,harper lee
4,F. Scott Fitzgerald,1925,The Great Gatsby,4,great gatsby,f scott fitzgerald


## Content-based Recommender

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [84]:
import numpy as np
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

# Ensure nltk data is downloaded (stopwords, punkt, wordnet)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [85]:

def prepare_tfidf(df):
    # Combine title and author after cleaning for more context
    df['combined'] = df['cleaned_title'] + " " + df['cleaned_author']
    # df['combined'] = df['combined'].apply(clean_text)
    
    tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined'])
    return tfidf_matrix, tfidf_vectorizer


In [86]:
import random

def generate_content_recommendations(df, query, top_k=5, similarity_threshold=0.1):

    tfidf_matrix, tfidf_vectorizer = prepare_tfidf(df)
    
    #transforming the cleaned query to match TF-IDF dimensions
    query_vector = tfidf_vectorizer.transform([clean_text(query)])
    
    #computing cosine similarity
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Sorting items based on similarity scores, but only consider those above the threshold
    above_threshold_indices = np.where(cosine_sim >= similarity_threshold)[0]

    top_indices = np.argsort(cosine_sim[above_threshold_indices]) # Fetch more items for diversity
    
    #times 2 for safety factor
    random_selection = random.choices(top_indices, k=(len(top_indices) % top_k**3))
    
    # #fetching top indices 
    # top_indices = np.argsort(cosine_sim)[-top_k*10:]  #fetch more items for diversity (*10)
    
    seen = set()
    unique_recommendations = []
    # Reverse because when we sort we get smallest to largest
    # print(len(random_selection))
    for index in random_selection:
        actual_index = above_threshold_indices[index]
        title_author = (df.iloc[actual_index]['Title'].lower(), df.iloc[actual_index]['Author'].lower(), df.iloc[actual_index]['Year'])

        same_as_query = df.iloc[actual_index]['Title'].lower() == query.lower()            

        if title_author not in seen and not same_as_query:
            seen.add(title_author)
            unique_recommendations.append(actual_index)

        # Continue adding until we truly have top_k unique or exhaust the list
        if len(unique_recommendations) == top_k or len(seen) == len(top_indices):
            break

        # if len(unique_recommendations) == top_k:
        #     break
    
    recommendations = df.iloc[unique_recommendations][['Title', 'Author', 'Year']]
    return recommendations  # Return recommendations in descending order of similarity
    # return recommendations



In [115]:
query_title = "love, rosie"
recommended_titles = generate_content_recommendations(bx, query_title, top_k=5)
print(recommended_titles)

                                                    Title             Author  \
142141                              Mama, Do You Love Me?  Barbara M. Joosse   
172548  The Angel and the Rake (Camfield Novel of Love...   Barbara Cartland   
8885                 When the One You Love Wants to Leave   Donald R. Harvey   
25650          Love Only Once (Malory Novels (Paperback))    Johanna Lindsey   
202488                              Scottish Love Stories      Susie Maguire   

        Year  
142141  1991  
172548  1993  
8885    1993  
25650   1985  
202488  1995  


## Collaborative-filtering Recommender (item-based memory-based approach)

In [87]:
id_bx.head()

,ISBN,Title,Author,Year,Publisher,UserID,BookRating,Scaled Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0.000000,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,4.928571,3
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,5.000000,3
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,4.272727,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0.000000,1


In [88]:
id_gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,UserID,Mean Rating
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,314,4
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,1169,4
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,314,3
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,439,4
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,314,4


### Cleaning 'Title' columns

In [89]:
# Apply the text_clean function to the 'Title' column of the first data frame 'bx' i.e., the Book Crossing dataset's final data frame
id_bx['cleaned_title'] = id_bx['Title'].apply(clean_text)

In [90]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - id_bx:")
print(id_bx[['Title', 'cleaned_title']].head())

First DataFrame - id_bx:
                                               Title  \
0                                Classical Mythology   
1                                       Clara Callan   
2                               Decision in Normandy   
3  Flu: The Story of the Great Influenza Pandemic...   
4                             The Mummies of Urumchi   

                                       cleaned_title  
0                                classical mythology  
1                                       clara callan  
2                                  decision normandy  
3  flu story great influenza pandemic search viru...  
4                                      mummy urumchi  


In [91]:
# Apply the text_clean function to the 'Title' column of the second data frame 'gbx' i.e., the Goodbooks-10k dataset's final data frame
id_gbx['cleaned_title'] = id_gbx['Title'].apply(clean_text)

In [92]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - id_gbx:")
print(id_gbx[['Title', 'cleaned_title']].head())


Second DataFrame - id_gbx:
                                      Title                   cleaned_title
0                          The Hunger Games                     hunger game
1  Harry Potter and the Philosopher's Stone  harry potter philosopher stone
2                                  Twilight                        twilight
3                     To Kill a Mockingbird                kill mockingbird
4                          The Great Gatsby                    great gatsby


### Cleaning 'Author' columns

In [93]:
# Apply the clean_text function to the 'Author' column of the first data frame 'bx'
id_bx['cleaned_author'] = id_bx['Author'].apply(clean_text)

In [94]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - bx:")
print(id_bx[['Author', 'cleaned_author']].head())

First DataFrame - bx:
                 Author        cleaned_author
0    Mark P. O. Morford        mark p morford
1  Richard Bruce Wright  richard bruce wright
2          Carlo D'Este           carlo deste
3      Gina Bari Kolata      gina bari kolata
4       E. J. W. Barber          e j w barber


In [95]:
# Apply the clean_text function to the 'Author' column of the second data frame 'gbx'
id_gbx['cleaned_author'] = id_gbx['Author'].apply(clean_text)

In [96]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - id_gbx:")
print(id_gbx[['Author', 'cleaned_author']].head())


Second DataFrame - id_gbx:
                         Author           cleaned_author
0               Suzanne Collins          suzanne collins
1  J.K. Rowling, Mary GrandPrÃ©  jk rowling mary grandpr
2               Stephenie Meyer          stephenie meyer
3                    Harper Lee               harper lee
4           F. Scott Fitzgerald       f scott fitzgerald


In [97]:
idbxcolumns_to_copy = ['ISBN', 'Publisher', 'BookRating']
id_bx.drop(columns=idbxcolumns_to_copy, inplace=True)
id_bx.head()

,Title,Author,Year,UserID,Scaled Rating,cleaned_title,cleaned_author
0,Classical Mythology,Mark P. O. Morford,2002,2,1,classical mythology,mark p morford
1,Clara Callan,Richard Bruce Wright,2001,8,3,clara callan,richard bruce wright
2,Decision in Normandy,Carlo D'Este,1991,8,3,decision normandy,carlo deste
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,8,3,flu story great influenza pandemic search viru...,gina bari kolata
4,The Mummies of Urumchi,E. J. W. Barber,1999,8,1,mummy urumchi,e j w barber


In [98]:
new_idgbxcolumns_to_copy = ['id', 'BookID', 'Avgrating', 'ratings_count']
id_gbx.drop(columns=new_idgbxcolumns_to_copy, inplace=True)
id_gbx = id_gbx.rename(columns={'Mean Rating': 'Scaled Rating'})
id_gbx.head()

,Author,Year,Title,UserID,Scaled Rating,cleaned_title,cleaned_author
0,Suzanne Collins,2008,The Hunger Games,314,4,hunger game,suzanne collins
1,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,1169,4,harry potter philosopher stone,jk rowling mary grandpr
2,Stephenie Meyer,2005,Twilight,314,3,twilight,stephenie meyer
3,Harper Lee,1960,To Kill a Mockingbird,439,4,kill mockingbird,harper lee
4,F. Scott Fitzgerald,1925,The Great Gatsby,314,4,great gatsby,f scott fitzgerald


In [99]:
id_bx.fillna(0, inplace=True)  # Replace NaN with 0

In [100]:
id_gbx.fillna(0, inplace=True)  # Replace NaN with 0

In [101]:
#combining the cleaned_title and cleaned_author columns to form a new column 
id_bx['combined'] = id_bx['cleaned_title'] + ' ' + id_bx['cleaned_author']
id_gbx['combined'] = id_gbx['cleaned_title'] + ' ' + id_gbx['cleaned_author']

In [102]:
id_bx.head()

,Title,Author,Year,UserID,Scaled Rating,cleaned_title,cleaned_author,combined
0,Classical Mythology,Mark P. O. Morford,2002,2,1,classical mythology,mark p morford,classical mythology mark p morford
1,Clara Callan,Richard Bruce Wright,2001,8,3,clara callan,richard bruce wright,clara callan richard bruce wright
2,Decision in Normandy,Carlo D'Este,1991,8,3,decision normandy,carlo deste,decision normandy carlo deste
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,8,3,flu story great influenza pandemic search viru...,gina bari kolata,flu story great influenza pandemic search viru...
4,The Mummies of Urumchi,E. J. W. Barber,1999,8,1,mummy urumchi,e j w barber,mummy urumchi e j w barber


In [103]:
id_gbx.head()

,Author,Year,Title,UserID,Scaled Rating,cleaned_title,cleaned_author,combined
0,Suzanne Collins,2008,The Hunger Games,314,4,hunger game,suzanne collins,hunger game suzanne collins
1,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,1169,4,harry potter philosopher stone,jk rowling mary grandpr,harry potter philosopher stone jk rowling mary...
2,Stephenie Meyer,2005,Twilight,314,3,twilight,stephenie meyer,twilight stephenie meyer
3,Harper Lee,1960,To Kill a Mockingbird,439,4,kill mockingbird,harper lee,kill mockingbird harper lee
4,F. Scott Fitzgerald,1925,The Great Gatsby,314,4,great gatsby,f scott fitzgerald,great gatsby f scott fitzgerald


In [104]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

In [105]:
def filter_and_create_user_item_matrix(df, min_book_ratings=3, min_user_ratings=3):
    #Filtering out sparse books and users, then creating a user-item matrix from the filtered dataframe.
  
    # Filtering out books having less than min_book_ratings
    book_counts = df['combined'].value_counts() #taking 'combined' column to consider every unique possible author&book combination
    books_to_keep = book_counts[book_counts >= min_book_ratings].index
    
    # Filtering out users having less than min_user_ratings
    user_counts = df['UserID'].value_counts()
    users_to_keep = user_counts[user_counts >= min_user_ratings].index
    
    # Filtering the resulting dataframe
    filtered_df = df[df['combined'].isin(books_to_keep) & df['UserID'].isin(users_to_keep)]
    
    # Creating the user-item matrix
    user_item_matrix = filtered_df.pivot_table(index='UserID', columns='combined', values='Scaled Rating', aggfunc='mean').fillna(0)
    
    # Converting to sparse matrix
    user_item_matrix_sparse = csr_matrix(user_item_matrix.values)
    
    return user_item_matrix, user_item_matrix_sparse


In [106]:
def generate_collaborative_recommendations(df, query, top_k=5, similarity_threshold=0.02):
    if 'combined' not in df.columns:
        df['combined'] = df['cleaned_title'] + " " + df['cleaned_author']
    
    cleaned_query = clean_text(query)
    
    matched_books = df[df['combined'].str.contains(cleaned_query, case=False, regex=False)]
    if matched_books.empty:
        return "No recommendations found."


    # Use the new function to filter sparse items/users and create the matrix
    user_item_matrix, user_item_matrix_sparse = filter_and_create_user_item_matrix(matched_books, min_book_ratings=1, min_user_ratings=1)
    if user_item_matrix.empty:
        return "No recommendations found as user-item matrix is empty after filtering."
    
    if user_item_matrix_sparse.shape[0] == 0 or user_item_matrix_sparse.shape[1] == 0:
        return "No recommendations found as user-item matrix is empty after filtering."

    item_similarity = cosine_similarity(user_item_matrix_sparse.T)

    matched_indices = []
    for book in matched_books['combined']:
        if book in user_item_matrix.columns:
            matched_indices.append(user_item_matrix.columns.get_loc(book))
    if not matched_indices:
        return "No recommendations found as no matches found in user-item matrix."


    aggregated_similarities = np.mean(item_similarity[matched_indices], axis=0)
    
    above_threshold_indices = np.where(aggregated_similarities >= similarity_threshold)[0]

    if len(above_threshold_indices) == 0:
        return "No recommendations found above the similarity threshold."

    top_indices = np.argsort(aggregated_similarities[above_threshold_indices])[-top_k**3:]  # Fetch more items for diversity
 
    unique_set = set(random.sample(list(top_indices), k=min(len(top_indices), top_k)))
    
    recommended_combined = []
    for i in unique_set:
        recommended_combined.append(user_item_matrix.columns[i])
        
    recommended_titles = df[df['combined'].isin(recommended_combined)][['Title', 'Author', 'Year']].drop_duplicates()

    # Ensure the number of recommendations does not exceed top_k
    if len(recommended_titles) > top_k:
        recommended_titles = recommended_titles.head(top_k)
    
    return recommended_titles

In [107]:
query_string = "sylvia plath"  # or any author's name
recommended_books = generate_collaborative_recommendations(id_gbx, query_string, top_k=5)
print(recommended_books)

                             Title                        Author  Year
175                   The Bell Jar                  Sylvia Plath  1963
2834                         Ariel                  Sylvia Plath  1965
3271           The Collected Poems      Sylvia Plath, Ted Hughes  1981
7240  The Journals of Sylvia Plath  Sylvia Plath, Karen V. Kukil  1982


In [374]:
# id_gbx[id_gbx['Title'].str.contains("Love")]

In [108]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [109]:
def interactive_recommendation_system(df):
    # Text field input for the query
    query_input = widgets.Text(
        value='',
        placeholder='Type book title or author',
        description='Name:',
        disabled=False
    )
    
    # Dropdown to select the recommendation type
    rec_type = widgets.Dropdown(
        options=['Select', 'Content-Based', 'Collaborative'],
        value='Select',
        description='Model:',
    )
    
    # Button to generate recommendations
    generate_button = widgets.Button(description="Generate Recommendations")
    
    # Output widget to display the recommendations
    output = widgets.Output()
    
    def on_generate_button_clicked(b):
        with output:
            clear_output()
            query = query_input.value
            top_k = 5  # You can modify this to be user-selectable as well
            similarity_threshold = 0.02  # Default threshold, adjust as necessary
            
            if query:  # Check if query is not empty
                if rec_type.value == 'Content-Based':
                    recommendations = generate_content_recommendations(df, query, top_k, similarity_threshold)
                else:  # Collaborative
                    recommendations = generate_collaborative_recommendations(df, query, top_k, similarity_threshold)
                
                if not recommendations.empty:
                    display(recommendations)
                else:
                    print("No recommendations found.")
            else:
                print("Please enter a query.")
    
    generate_button.on_click(on_generate_button_clicked)
    
    # Display the widgets
    display(query_input, rec_type, generate_button, output)

In [119]:
# Example usage (make sure `df` is defined as your DataFrame)
interactive_recommendation_system(id_bx)

Text(value='', description='Name:', placeholder='Type book title or author')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative'), value='Select')

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [444]:
# def generate_user_similar_collab_recommendations(df, query, top_k=5, similarity_threshold=0.02):
#     if 'combined' not in df.columns:
#         df['combined'] = df['cleaned_title'] + " " + df['cleaned_author']
    
#     # Find books that match the query
#     query_matched_books = df[df['combined'].str.contains(query, case=False, regex=False)]
#     if query_matched_books.empty:
#         return "No matching books or authors found."
    
#     # Filter the DataFrame to users who have interacted with the matched books
#     users_interacted = query_matched_books['UserID'].unique()
    
#     # Filter and create user-item matrix including only users who have interacted with the matched books
#     user_item_matrix, user_item_matrix_sparse = filter_and_create_user_item_matrix(df[df['UserID'].isin(users_interacted)], min_book_ratings=3, min_user_ratings=3)
    
#     # Calculate cosine similarity across users
#     user_similarity = cosine_similarity(user_item_matrix_sparse)
    
#     # Find indices of users who have interacted with the query in the user-item matrix
#     interacted_user_indices = [user_item_matrix.index.get_loc(user_id) for user_id in users_interacted]
    
#     # Calculate mean similarity score of each user to the users who interacted with the query
#     mean_similarity_scores = np.mean(user_similarity[:, interacted_user_indices], axis=1)
    
#     # Apply similarity threshold
#     similar_users_indices = np.where(mean_similarity_scores >= similarity_threshold)[0]
    
#     if len(similar_users_indices) == 0:
#         return "No similar users found above the similarity threshold."
    
#     # Aggregate book ratings from similar users
#     recommended_books = {}
#     for idx in similar_users_indices:
#         similar_user_id = user_item_matrix.index[idx]
#         for book, rating in user_item_matrix.loc[similar_user_id].items():
#             if pd.notna(rating) and rating > 0:  # Assuming a rating > 0 indicates a positive interaction
#                 if book not in recommended_books:
#                     recommended_books[book] = [rating]
#                 else:
#                     recommended_books[book].append(rating)
    
#     # Sort books by the average rating and select top_k
#     sorted_recommended_books = sorted(recommended_books.items(), key=lambda x: np.mean(x[1]), reverse=True)[:top_k]
    
#     # Prepare the final DataFrame of recommendations
#     recommended_titles = pd.DataFrame(sorted_recommended_books, columns=['Book', 'Average Rating'])
    
#     # Since 'Book' contains the 'combined' column, split to show title and author separately if needed
#     recommended_titles[['Title', 'Author']] = recommended_titles['Book'].str.split(" ", 1, expand=True)
    
#     return recommended_titles[['Title', 'Author', 'Average Rating']]


In [ ]:
# # Example usage:
# # df should be your DataFrame containing book interactions
# # query can be either a book title or an author's name
# # top_k is the number of recommendations you want
# # similarity_threshold is the threshold for considering user similarity
# query = 'Harry Potter'
# recommendations = generate_user_similar_collab_recommendations(id_bx, query, top_k=5, similarity_threshold=0.1)
# print(recommendations)

## Hybrid-filtering Recommendation Model

### Switching Hybrid Model

In [122]:
def generate_switching_hybrid_recommendations(df, query, top_k=5, similarity_threshold=0.1):
    # Check if the query matches enough user interactions for collaborative filtering
    if len(df[df['combined'].str.contains(query, case=False)]) > similarity_threshold:
        return generate_collaborative_recommendations(df, query, top_k, similarity_threshold)
    else:
        return generate_content_recommendations(df, query, top_k, similarity_threshold)

In [126]:
query = "Love, Rosie"
generate_switching_hybrid_recommendations(id_bx, query, top_k=5)

,Title,Author,Year
234698,"The Power That Heals: Love, Healing &amp; the ...",David James Randolph,1994
225531,Love Invents Us,Amy Bloom,2002
262429,A Puppy to Love (Clifford's Big Ideas),Bob Barkly,2001
186775,"Dangerous To Love (Devil's Gambit, #5)",Lisa Jackson,1985
122485,Love Lives: Why Women Behave the Way They Do i...,Carol Bowtin,1983


In [127]:
query = "Love, Rosie"
generate_switching_hybrid_recommendations(id_gbx, query, top_k=5)

,Title,Author,Year
5764,"Fables, Volume 3: Storybook Love","Bill Willingham, Mark Buckingham, Steve Leialo...",2004
6204,Passion and Purity: Learning to Bring Your Lov...,Elisabeth Elliot,1984
1108,Ugly Love,Colleen Hoover,2014
3698,"Conversations with God, An Uncommon Dialogue: ...",Neale Donald Walsch,1995
7408,"Lunch in Paris: A Love Story, with Recipes",Elizabeth Bard,2010


In [128]:
query = "Love, Rosie"
generate_switching_hybrid_recommendations(bx, query, top_k=5)

,Title,Author,Year
138298,Reflections of Love,Alice Joyce Davidson,1983
43039,NAPOLEON AND LOVE,Philip MacKie,1976
222293,Lesbian Love Stories,Irene Zahava,1989
143794,First Loves : Poets Introduce the Essential Po...,Carmela Ciuraru,2000
169879,Love for Sale. Erotische Fantasien.,Marie Sahr,2002


In [129]:
query = "Love, Rosie"
generate_switching_hybrid_recommendations(gbx, query, top_k=5)

,Title,Author,Year
642,"I'd Tell You I Love You, But Then I'd Have to ...",Ally Carter,2006
7486,"What Remains: A Memoir of Fate, Friendship, an...",Carole Radziwill,2005
9404,The Mambo Kings Play Songs of Love,Oscar Hijuelos,1989
7283,of course i love you,"Durjoy Datta, Maanvi Ahuja",2008
7463,The Love Song of J. Alfred Prufrock and Other ...,T.S. Eliot,1915


In [133]:
query = "The Fault In our stars"
generate_switching_hybrid_recommendations(bx, query, top_k=5)

,Title,Author,Year
16217,Star Wars Episode I: The Phantom Menace,Patricia C. Wrede,1999
37825,Life Signs: The Biology of Star Trek,Susan Jenkins,1998
205548,Follow the Star Pushalong Book,Pushalong Book,2001
214152,DWELLERS IN CRUCBL (Star Trek (Numbered Paperb...,Margaret Wander Bonanno,1988
167107,LONE STAR RANGER,Zane Grey,1984


### Feature Combination Hybrid Model

In [134]:
def generate_feature_combination_hybrid_recommendations(df, query, top_k=5, similarity_threshold=0.1):
    # Example: Assume a machine learning model is trained on features combining content attributes and user interactions
    # For demonstration, we'll manually combine scores from both methods
    
    content_recs = generate_content_recommendations(df, query, top_k, similarity_threshold)
    collaborative_recs = generate_collaborative_recommendations(df, query, top_k, similarity_threshold)
    
    # Assume both functions return DataFrames with 'Title', 'Author', and 'Score'
    # Merge the recommendations on 'Title' and 'Author', combining scores
    combined_recs = pd.merge(content_recs, collaborative_recs, on=['Title', 'Author'], how='outer', suffixes=('_content', '_collab'))
    
    # Fill missing values and combine scores
    combined_recs.fillna(0, inplace=True)
    combined_recs['combined_score'] = combined_recs['Score_content'] + combined_recs['Score_collab']
    
    # Sort and select top recommendations
    final_recs = combined_recs.sort_values(by='combined_score', ascending=False).head(top_k)
    
    return final_recs[['Title', 'Author', 'combined_score']]

In [ ]:
query = "Love, Rosie"
generate_feature_combination_hybrid_recommendations(id_bx, query, top_k=5)

### Cascade Hybrid Model

In [136]:
def generate_cascade_hybrid_recommendations(df, query, top_k=5, intermediate_k=10, similarity_threshold=0.1):
    # Generate an intermediate set of recommendations using content-based filtering
    intermediate_recs = generate_content_recommendations(df, query, intermediate_k, similarity_threshold)
    
    # Refine the intermediate set using collaborative filtering
    # For this example, assume collaborative recommendations can accept a list of books
    final_recs = generate_collaborative_recommendations(df, intermediate_recs['Title'].tolist(), top_k, similarity_threshold)
    
    return final_recs


In [137]:
query = "Love, Rosie"
generate_cascade_hybrid_recommendations(id_bx, query, top_k=5)

AttributeError: 'list' object has no attribute 'lower'